In [2]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
from six import BytesIO
from glob import glob
from PIL import Image

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

%matplotlib inline

In [3]:
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
  #(im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((1024,1024,3)).astype(np.uint8)

category_index = {
    1: {'id': 1, 'name': 'zebra cross'}
}

In [4]:
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('saved_model/')
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

Elapsed time: 8.344902038574219s


In [8]:
paths = glob('test/*.jpg')

for path in paths:
    image_np = load_image_into_numpy_array(path)
    input_tensor = np.expand_dims(image_np, 0)
    
    detections = detect_fn(input_tensor)
    
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.31,
        agnostic_mode=False)
    result = Image.fromarray(image_np_with_detections)
    result.save('prediction/'+path.split('/')[2])